In [19]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.preprocessing import MinMaxScaler

In [7]:
model = Sequential()
model.add(Dense(32, input_dim=10, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(loss="mean_squared_error", optimizer='adam')

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 1,025
Trainable params: 1,025
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.load_weights('./Model_check_point/model_checkpoint_20210712_1913.ckpt')

In [14]:
data = pd.read_csv('./data/data_frame_v2.csv', index_col=None)
print(data.columns)
data

Index(['year', 'month', 'ID', 'data_block_num', 'shop_id', 'item_id',
       'item_category_id', 'item_price', 'category_occ',
       'total_mean_item_price', 'item_cnt_month'],
      dtype='object')


,year,month,ID,data_block_num,shop_id,item_id,item_category_id,item_price,category_occ,total_mean_item_price,item_cnt_month
0,2013,1,5122,0,4,5823,35,2500.00,162,2635.116266,1
1,2013,1,5127,0,4,5643,2,2330.25,43,2539.082418,3
2,2013,1,5142,0,4,5821,35,599.00,162,822.133789,2
3,2013,1,5143,0,4,5822,35,999.00,162,1064.279381,9
4,2013,1,5152,0,4,5459,55,299.00,1105,330.464615,1
...,...,...,...,...,...,...,...,...,...,...,...
600154,2015,10,213962,33,45,16106,64,799.00,158,573.999197,1
600155,2015,10,214068,33,45,6184,30,498.00,1362,670.072416,1
600156,2015,10,214069,33,45,3985,55,98.00,1105,249.491071,1
600157,2015,10,214187,33,45,5401,19,799.00,677,1626.173585,1


In [13]:
sample = pd.read_csv('./data/sample_submission.csv', index_col=None)
sample

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [100]:
# y_test에 zero-padding
y_test = sample[['ID']].copy()
y_test = pd.merge(y_test, data[data.data_block_num == 33][['ID', 'item_cnt_month']], how='left', on='ID')
y_test.fillna(0, inplace=True)

# y_pred에 zero-padding
x_train = data[data.data_block_num < 33].values[:, :10]
x_test = data[data.data_block_num == 33].values[:, :10]

scaler = MinMaxScaler()
scaler.fit(x_train)
x_test_scale = scaler.transform(x_test)

y_pred_raw = model.predict(x_test_scale)
y_temp = data[data.data_block_num == 33][['ID', 'item_cnt_month']].copy()
y_temp['item_cnt_month'] = y_pred_raw

y_pred = sample[['ID']].copy()
y_pred = pd.merge(y_pred, y_temp, how='left', on='ID')
y_pred.fillna(0, inplace=True)
y_pred

,ID,item_cnt_month
0,0,0.000000
1,1,0.000000
2,2,0.385627
3,3,0.000000
4,4,0.000000
...,...,...
214195,214195,0.524233
214196,214196,0.000000
214197,214197,0.000000
214198,214198,0.000000


In [26]:
# RMSE 계산하기
y_t = y_test[['item_cnt_month']].values
y_p = y_pred[['item_cnt_month']].values
rmse = (((y_t - y_p) ** 2).mean()) ** 0.5
print(rmse)

5.426002704776644


In [ ]:
# 예측 값과 실제 값의 비교
y_pred = model.predict(x_test_scale).flatten()
rmse = (((y_test - y_pred) ** 2).mean()) ** 0.5
print(rmse)
print("Mission Complete!!!")

# 이번엔 어느 상품이 팔릴지 모른다고 가정하고 RMSE 출력해보기

* 최근 3개월동안 팔린 item
* 같은 월에 팔린 item

In [69]:
sell = sample.copy()
sell.item_cnt_month = 0
sell.columns = ['ID', 'sell']

# 3개월 동안 팔렸나 확인
data_3m = data[(30 <= data.data_block_num) & (data.data_block_num <= 32)][['data_block_num', 'ID', 'item_cnt_month']]
data_3m = data_3m.groupby('ID').sum()[['item_cnt_month']].reset_index()

sell = pd.merge(sell, data_3m,
                how='left', on='ID')
sell.fillna(0, inplace=True)

sell['sell_3m'] = sell['item_cnt_month'].apply(lambda c: 1 if c>=3 else 0)

sell = sell[['ID', 'sell', 'sell_3m']]

# 같은 월에 팔린 item
data_same = data[data.month == 10].copy()
data_same = data_same[data_same.year < 2015]
data_same = data_same.groupby('ID').sum()[['item_cnt_month']].reset_index()

sell = pd.merge(sell, data_same,
                how='left', on='ID')
sell.fillna(0, inplace=True)

sell['sell_same'] = sell['item_cnt_month'].apply(lambda c: 1 if c>=2 else 0)

sell = sell[['ID', 'sell', 'sell_3m', 'sell_same']]

# 합집합 남기기
sell['sell'] = sell['sell_3m'] + sell['sell_same']
sell['sell'] = sell['sell'].apply(lambda c: 1 if c>0 else 0)

sell = sell[['ID', 'sell']]
sell

,ID,sell
0,0,1
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
214195,214195,0
214196,214196,0
214197,214197,0
214198,214198,0


In [64]:
please1 = data[(30 <= data.data_block_num) & (data.data_block_num <= 32)]
please2 = data[data.month == 10]
please2 = please2[please2.year < 2015]
please = pd.concat([please1, please2])
please

,year,month,ID,data_block_num,shop_id,item_id,item_category_id,item_price,category_occ,total_mean_item_price,item_cnt_month
489587,2015,7,0,30,5,5037,19,1499.0,677,1891.068359,1
489588,2015,7,8,30,5,5319,55,299.0,1105,298.372458,3
489589,2015,7,10,30,5,4806,30,99.0,1362,144.479240,5
489590,2015,7,13,30,5,4869,22,1499.0,64,1257.927595,1
489591,2015,7,14,30,5,4870,23,866.0,569,914.424235,4
...,...,...,...,...,...,...,...,...,...,...,...
290809,2014,10,214068,21,45,6184,30,699.0,1362,670.072416,7
290810,2014,10,214103,21,45,15472,63,549.0,177,635.375066,1
290811,2014,10,214112,21,45,12569,37,499.0,350,446.182857,1
290812,2014,10,214151,21,45,7925,7,2999.0,28,2884.905856,4


In [70]:
please = please.groupby('ID').mean()[['item_price']].reset_index()
please

,ID,item_price
0,0,1082.555556
1,2,799.000000
2,3,599.000000
3,5,749.500000
4,6,3832.333333
...,...,...
65716,214178,899.000000
65717,214179,599.000000
65718,214181,899.000000
65719,214185,149.000000


In [72]:
sell = pd.merge(sell, please, how='left', on='ID')
sell.fillna(0, inplace=True)
sell

,ID,sell,item_price
0,0,1,1082.555556
1,1,0,0.000000
2,2,1,799.000000
3,3,0,599.000000
4,4,0,0.000000
...,...,...,...
214195,214195,0,0.000000
214196,214196,0,0.000000
214197,214197,0,0.000000
214198,214198,0,0.000000


In [73]:
data

,year,month,ID,data_block_num,shop_id,item_id,item_category_id,item_price,category_occ,total_mean_item_price,item_cnt_month
0,2013,1,5122,0,4,5823,35,2500.00,162,2635.116266,1
1,2013,1,5127,0,4,5643,2,2330.25,43,2539.082418,3
2,2013,1,5142,0,4,5821,35,599.00,162,822.133789,2
3,2013,1,5143,0,4,5822,35,999.00,162,1064.279381,9
4,2013,1,5152,0,4,5459,55,299.00,1105,330.464615,1
...,...,...,...,...,...,...,...,...,...,...,...
600154,2015,10,213962,33,45,16106,64,799.00,158,573.999197,1
600155,2015,10,214068,33,45,6184,30,498.00,1362,670.072416,1
600156,2015,10,214069,33,45,3985,55,98.00,1105,249.491071,1
600157,2015,10,214187,33,45,5401,19,799.00,677,1626.173585,1


In [75]:
test_csv = pd.read_csv('./data/test.csv', index_col=None)
test_csv

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [77]:
items_csv = pd.read_csv('./data/items.csv', index_col=None)
items_csv

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


In [134]:
occ_csv = pd.read_csv('./data/occ.csv', index_col=None)
occ_csv.columns = ['item_category_id', 'category_occ']
occ_csv.head()

,item_category_id,category_occ
0,2,43.0
1,3,119.0
2,5,13.0
3,6,84.0
4,7,28.0


In [88]:
tmip = pd.read_csv('./data/total_mean_item_price.csv', index_col=None)
tmip.columns = ['item_id', 'total_mean_item_price']
tmip.head()

,item_id,total_mean_item_price
0,30,324.562531
1,31,573.793088
2,32,250.083706
3,33,249.273016
4,38,2351.537037


In [135]:
x_test2 = sell[sell.sell==1].copy()
x_test2 = x_test2[['ID', 'item_price']]
x_test2['year'] = 2015
x_test2['month'] = 10
x_test2['data_block_num'] = 33

x_test2 = pd.merge(x_test2, test_csv, how='left', on='ID')
x_test2 = pd.merge(x_test2, items_csv[['item_id', 'item_category_id']], how='left', on='item_id')

x_test2 = pd.merge(x_test2, occ_csv, how='left', on='item_category_id')
x_test2 = pd.merge(x_test2, tmip, how='left', on='item_id')

x_test2

,ID,item_price,year,month,data_block_num,shop_id,item_id,item_category_id,category_occ,total_mean_item_price
0,0,1082.555556,2015,10,33,5,5037,19,677.0,1891.068359
1,2,799.000000,2015,10,33,5,5233,19,677.0,802.390632
2,6,3832.333333,2015,10,33,5,5041,20,606.0,3837.086434
3,8,299.000000,2015,10,33,5,5319,55,1105.0,298.372458
4,10,149.000000,2015,10,33,5,4806,30,1362.0,144.479240
...,...,...,...,...,...,...,...,...,...,...
24760,213951,1998.850000,2015,10,33,45,2810,23,569.0,1916.514557
24761,213999,599.000000,2015,10,33,45,17010,55,1105.0,590.716220
24762,214041,599.000000,2015,10,33,45,6488,28,387.0,514.850994
24763,214068,699.000000,2015,10,33,45,6184,30,1362.0,670.072416


In [136]:
# x_test2 = x_test[data.columns.to_list()[:-1]]
# print(x_test2.columns.to_list())
# print()
# print(data.columns)
col2 = data.columns.to_list()[:10]
x_test2 = x_test2[col2]

In [137]:
x_test2

,year,month,ID,data_block_num,shop_id,item_id,item_category_id,item_price,category_occ,total_mean_item_price
0,2015,10,0,33,5,5037,19,1082.555556,677.0,1891.068359
1,2015,10,2,33,5,5233,19,799.000000,677.0,802.390632
2,2015,10,6,33,5,5041,20,3832.333333,606.0,3837.086434
3,2015,10,8,33,5,5319,55,299.000000,1105.0,298.372458
4,2015,10,10,33,5,4806,30,149.000000,1362.0,144.479240
...,...,...,...,...,...,...,...,...,...,...
24760,2015,10,213951,33,45,2810,23,1998.850000,569.0,1916.514557
24761,2015,10,213999,33,45,17010,55,599.000000,1105.0,590.716220
24762,2015,10,214041,33,45,6488,28,599.000000,387.0,514.850994
24763,2015,10,214068,33,45,6184,30,699.000000,1362.0,670.072416


In [138]:
# y_test에 zero-padding
y_test = sample[['ID']].copy()
y_test = pd.merge(y_test, data[data.data_block_num == 33][['ID', 'item_cnt_month']], how='left', on='ID')
y_test.fillna(0, inplace=True)



In [139]:
# y_pred2에 zero-padding
x_test2_scale = scaler.transform(x_test2.values)

y_pred2_raw = model.predict(x_test2_scale)
y_temp2 = x_test2[['ID']].copy()
y_temp2['item_cnt_month'] = y_pred2_raw

y_pred2 = sample[['ID']].copy()
y_pred2 = pd.merge(y_pred2, y_temp2, how='left', on='ID')
y_pred2.fillna(0, inplace=True)
y_pred2.head(20)

,ID,item_cnt_month
0,0,2.556236
1,1,0.000000
2,2,0.615261
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,4.534632
7,7,0.000000
8,8,1.848270
9,9,0.000000


In [140]:
pd.merge(y_pred, y_pred2, how='left', on='ID').head(20)

,ID,item_cnt_month_x,item_cnt_month_y
0,0,0.000000,2.556236
1,1,0.000000,0.000000
2,2,0.385627,0.615261
3,3,0.000000,0.000000
4,4,0.000000,0.000000
5,5,2.815304,0.000000
6,6,4.611660,4.534632
7,7,0.000000,0.000000
8,8,0.000000,1.848270
9,9,0.000000,0.000000


In [141]:
# RMSE 계산하기
y_t = y_test[['item_cnt_month']].values
y_p2 = y_pred2[['item_cnt_month']].values
rmse2 = (((y_t - y_p2) ** 2).mean()) ** 0.5
print(rmse2)

5.536150122845464


In [142]:
rmse

5.426002704776644